# Celgene - Accumulations

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("switzerland-efpia-documents-eng-20171.pdf", pages='all', lattice=True)

In [3]:
df_export = df.copy()

#Shift
column0 = df_export[df_export.columns[0]]
df_export[column0.isna()] = df_export[column0.isna()].shift(-1, axis='columns')

#Drop Columns
df_export = df_export.drop(df_export.columns[9:],axis=1)
df_export = df_export.drop(df_export.columns[[7]],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount', na=False)]
recipients = df_export[df_export.type.str.contains('Number of Recipients', na=False)]
rnd = df_export[df_export.type.str.contains('Transfers of Value', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Numberize
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"
df_export.loc[df_export.type == 'rnd', "total"] = df_export['donations_grants']
df_export.loc[df_export.type == 'rnd', "donations_grants"] = np.NaN

#Export
export_acumulations(df_export, 'celgene')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')